# Tree with state propagation

In [1]:
%matplotlib widget
import bmcs_utils.api as bu
import traits.api as tr
from bmcs_utils.model import Model

In [2]:
scd = False

Each model can be regarded as a tree of model components.
Let us define models `TopModel`, `InterimModel` and `SubModel`.

In [3]:
class SubModel(Model):
    name = 'submodel'
    
    length = bu.Float(2, GEO=True)

    ipw_view = bu.View(
        bu.Item('length')
    )

In [4]:
sm = SubModel(state_change_debug=scd)

In [5]:
sm.length = 3

Let us define a more complex model

In [6]:
class InterimModel(Model):
    sm = bu.Instance(SubModel,())
    
    stiffness = tr.Property(bu.Float, depends_on='state_changed')
    @tr.cached_property
    def _get_stiffness(self):
        return self.sm.length * 10
    
    depends_on = ['sm']
    
    ipw_view = bu.View(
        bu.Item('stiffness', readonly=True)
    )

In [7]:
im = InterimModel(sm=sm, state_change_debug=scd)
im.children

['sm']

In [8]:
im.sm.length = 8

In [9]:
class TopLevelModelEagerGraphChange(Model):
    name = 'tm'
    
    sm = bu.Instance(SubModel, ())
    def _sm_default(self):
        sm = SubModel()
        self.set_sm(sm)
        return sm
    def _sm_changed(self):
        self.set_sm(self.sm)
    def set_sm(self, sm):
        self.im.sm = sm
        self.im2.sm = sm
        
    im = bu.Instance(InterimModel, ())
    im2 = bu.Instance(InterimModel, ())

    time = bu.Float(400, ALG=True)

    tree = ['sm', 'im', 'im2']


In [10]:
tl = TopLevelModelEagerGraphChange(state_change_debug=scd)
tl.im.state_change_debug = scd
tl.im2.state_change_debug = scd
tl.im.sm.state_change_debug = scd
tl, tl.im, tl.im2, tl.sm.parents

(<__main__.TopLevelModelEagerGraphChange at 0x7f0babb00bd0>,
 {<__main__.InterimModel at 0x7f0bab76d4a0>,
  <__main__.TopLevelModelEagerGraphChange at 0x7f0babb00bd0>})

In [11]:
tl.sm = SubModel(state_change_debug=scd)
tl.sm, tl.im2.sm, tl.im.sm

(<__main__.SubModel at 0x7f0bac748220>,
 <__main__.SubModel at 0x7f0bac748220>)

In [12]:
tl.im.parents, tl.im2.parents, tl.sm.parents

({<__main__.TopLevelModelEagerGraphChange at 0x7f0babb00bd0>},
 {<__main__.TopLevelModelEagerGraphChange at 0x7f0babb00bd0>},
 {<__main__.InterimModel at 0x7f0bab76d4a0>,
  <__main__.TopLevelModelEagerGraphChange at 0x7f0babb00bd0>})

In [13]:
tl.im.sm.length = 10

In [14]:
tl.im, tl.im2, tl.im.sm.parents

(<__main__.InterimModel at 0x7f0bab76d4a0>,
 {<__main__.InterimModel at 0x7f0bab76d4a0>,
  <__main__.TopLevelModelEagerGraphChange at 0x7f0babb00bd0>})

In [15]:
tl.interact()

In [16]:
im.sm.length

8.0

In [17]:
class TopLevelModelLazyGraphChange(Model):
    name = 'tm'
    
    sm = bu.Instance(SubModel, ())
        
    im = tr.Property(bu.Instance(InterimModel), depends_on='sm')
    @tr.cached_property
    def _get_im(self):
        print('im')
        return InterimModel(sm=self.sm)

    im2 = tr.Property(bu.Instance(InterimModel), depends_on='sm')
    @tr.cached_property
    def _get_im2(self):
        print('im2')
        return InterimModel(sm=self.sm)

    time = bu.Float(400, ALG=True)

    tree = ['sm', 'im', 'im2']

In [18]:
tl = TopLevelModelEagerGraphChange()
tl.sm.parents, tl.children_traits, tl.im.children_traits

({<__main__.InterimModel at 0x7f0ba3625130>,
  <__main__.TopLevelModelEagerGraphChange at 0x7f0baba31ef0>},
 {'sm': <__main__.SubModel at 0x7f0ba36990e0>,
  'im': <__main__.InterimModel at 0x7f0ba369a360>,
  'im2': <__main__.InterimModel at 0x7f0ba3625130>},
 {'sm': <__main__.SubModel at 0x7f0bab76dae0>})

In [19]:
tl.interact()

Add change the component and check if the changes of stiffness in InterimModel get updated

In [28]:
tl.sm = SubModel()
tl.sm.length = 30
tl.im.stiffness

300.0